# Analyzing Pandas Serialization Formats

In [1]:
import pandas as pd
import numpy as np
import random
import time
import os
from tqdm.notebook import tqdm

In [2]:
def generate(rows=1000, cols=10, min_value=0, max_value=1000):
    columns = list(range(1, cols + 1))
    df = pd.DataFrame(data=np.random.randint(min_value, max_value,
                                             size=(rows, cols)),
                      columns=columns)
    return df

In [3]:
df = generate(rows=1000, cols=10)
df.shape

(1000, 10)

In [4]:
df.head()

,1,2,3,4,5,6,7,8,9,10
0,952,215,567,750,48,75,392,699,290,473
1,517,283,941,861,910,817,650,41,302,778
2,41,758,159,939,795,587,496,510,206,880
3,432,127,600,32,771,270,197,853,705,647
4,857,848,754,985,145,302,682,252,342,701


In [5]:
formats = ['csv', 'pickle', 'parquet', 'hdf', 'json', 'feather'] # xml
df_sizes = [1_000, 2_000, 5_000, 7_000, 10_000, 20_000, 50_000,
            100_000, 300_000, 500_000, 1_000_000, 2_000_000, 5_000_000,
            10_000_000, 50_000_000]
metrics_columns = ['df_size', 'format', 'saving_time', 'loading_time', 'file_size']

df_metrics = pd.DataFrame(data=None, columns=metrics_columns, dtype="object")
df_metrics

,df_size,format,saving_time,loading_time,file_size


In [6]:
def get_metrics(df_format, size, df):
    metrics = [size, df_format]
    
    file_path = "../data/test.obj"
    
    if df_format == 'hdf':
        file_path += '.h5'

    ## SAVING
    t0 = time.time()
    
    if df_format == 'csv':
        df.to_csv(file_path, index=False)
    elif df_format == 'pickle':
        df.to_pickle(file_path)
    elif df_format == 'parquet':
        df.to_parquet(file_path)
    elif df_format == 'hdf':
        df.to_hdf(file_path, key='main')
    elif df_format == 'json':
        df.to_json(file_path)
    elif df_format == 'xml':
        df.to_xml(file_path)
    elif df_format == 'feather':
        df.to_feather(file_path)
    elif df_format == 'html':
        df.to_html(file_path)
    else:
        print("Undefined format")
        
    t1 = time.time()
    saving_time = t1 - t0

    metrics.append(saving_time)

    
    ## LOADING
    t0 = time.time()
    
    if df_format == 'csv':
        df = pd.read_csv(file_path)
    elif df_format == 'pickle':
        df = pd.read_pickle(file_path)
    elif df_format == 'parquet':
        df = pd.read_parquet(file_path)
    elif df_format == 'hdf':
        df = pd.read_hdf(file_path, key='main')
    elif df_format == 'json':
        df = pd.read_json(file_path)
    elif df_format == 'xml':
        df = pd.read_xml(file_path)
    elif df_format == 'feather':
        df = pd.read_feather(file_path)
    elif df_format == 'html':
        df = pd.read_html(file_path)
    else:
        print("Undefined format")
    
    t1 = time.time()
    loading_time = t1 - t0

    metrics.append(loading_time)

    ## FILE SIZE (Converting Bytes to MegaBytes MB)
    file_size = (os.stat(file_path).st_size) / (1024 ** 2)
    metrics.append(file_size)
    
    ## DELETING FILE
    os.remove(file_path)
    
    return metrics
    

In [7]:
total_pbar = len(formats) * len(df_sizes)
with tqdm(total=total_pbar) as pbar:
    
    for size in df_sizes:
        df = generate(rows=size, cols=10)
        
        for serial_format in formats:
            pbar.set_description(f"Processing {serial_format} - {size:,}...")
            
            metrics = get_metrics(serial_format, size, df)
            df_metrics_records = pd.DataFrame(data=[metrics], columns=metrics_columns)
            
            if df_metrics.shape[0] == 0:
                df_metrics = df_metrics_records.copy()
            else:
                df_metrics = pd.concat([df_metrics, df_metrics_records])
                
            pbar.update(1)

df_metrics.reset_index(drop=True, inplace=True)

  0%|          | 0/90 [00:00<?, ?it/s]

In [8]:
df_metrics.describe()

,df_size,saving_time,loading_time,file_size
count,9.000000e+01,90.000000,90.000000,90.000000
mean,4.599667e+06,1.555326,12.540181,288.443238
std,1.248729e+07,6.683785,102.265847,970.674937
min,1.000000e+03,0.000540,0.000191,0.037146
25%,7.000000e+03,0.003534,0.002160,0.393219
50%,1.000000e+05,0.012526,0.010556,4.883303
75%,2.000000e+06,0.166305,0.113043,75.770025
max,5.000000e+07,48.758010,967.741834,6994.139887


In [9]:
df_metrics[df_metrics["df_size"] == 10_000_000].sort_values(by=["loading_time"])

,df_size,format,saving_time,loading_time,file_size
80,10000000,parquet,1.589181,0.080382,119.826619
79,10000000,pickle,0.105600,0.113706,762.940105
83,10000000,feather,0.455354,0.185272,299.299990
81,10000000,hdf,0.127229,0.347097,839.240303
78,10000000,csv,9.801016,2.106891,370.984526
82,10000000,json,5.798307,82.808826,1314.062510


In [10]:
df_metrics.to_csv("../data/pandas_metrics.csv", index=False)

In [11]:
df_metrics

,df_size,format,saving_time,loading_time,file_size
0,1000,csv,0.003857,0.001250,0.037146
1,1000,pickle,0.000684,0.000250,0.076942
2,1000,parquet,0.012529,0.009130,0.042720
3,1000,hdf,0.058311,0.002526,0.090828
4,1000,json,0.001124,0.004246,0.093356
...,...,...,...,...,...
85,50000000,pickle,0.720383,0.709117,3814.697918
86,50000000,parquet,8.203782,0.534680,599.039191
87,50000000,hdf,0.823863,3.080111,4196.173897
88,50000000,json,38.705391,967.741834,6994.139887


## Visualization